In [17]:
%load_ext autoreload
%autoreload 2

%load_ext memory_profiler
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [18]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# from helper import groupby_consecutive

import sys
sys.path.append('..')
from plotly_resampler import FigureResampler, EveryNthPoint, EfficientLTTB

# Basic sine example

In [19]:
n = 5_000_000
x = np.arange(n)
noisy_sine = (3 + np.sin(x / 2000) + np.random.randn(n) / 10) * x / 1000

# 1. Wrap the figure with the FigureResampler class
fig = FigureResampler(go.Figure())
# Add the trace using the `hf_x` & `hf_y` for faster rendering 
# (this does not impact the resampling speed once rendered)
fig.add_trace(go.Scattergl(), hf_x=x, hf_y=noisy_sine)
# fig.add_trace(go.Scattergl(x=x, y=noisy_sine))

# Optional: update the layout
fig.update_layout(height=400, template='plotly_dark')

# 2. Call show_dash
fig.show_dash(mode='inline')

# The example `.gif` from the docs/README

Note how the example figure withholds
* time-indexed data
* numeric-indexed data


In [20]:
df_gusb = pd.read_parquet("data/df_gusb.parquet")
df_data_pc = pd.read_parquet("data/df_pc_test.parquet")

# Create a noisy sine
n = 110_000_00#0
x = np.arange(n)
noisy_sine = (3 + np.sin(x / 200_000) + np.random.randn(n) / 10) * x / 100_000


In [21]:
fig = FigureResampler(
    make_subplots(
        rows=2,
        cols=2,
        specs=[[{}, {}], [{"colspan": 2}, None]],
        subplot_titles=("GUSB swimming pool", "Generated sine", "Power consumption"),
        vertical_spacing=0.12,
    ),
    default_n_shown_samples=1000,
    verbose=False,
)


# ------------ swimming pool data -----------
df_gusb_pool = df_gusb['zwembad'].last("4D").dropna()
fig.add_trace(
    go.Scattergl(
        x=df_gusb_pool.index,
        y=df_gusb_pool.astype("uint16"),
        mode="markers",
        marker_size=5,
        name="occupancy",
        # showlegend=False,
    ),
    hf_hovertext="mean last hour: "
    + df_gusb_pool.rolling("1h").mean().astype(int).astype(str),
    downsampler=EveryNthPoint(interleave_gaps=False),
    row=1,
    col=1,
)
fig.update_yaxes(title_text="Occupancy", row=1, col=1)


# ----------------- generated sine -----------
fig.add_trace(
    go.Scattergl(name="sin", line_color="#26b2e0"),
    hf_x=x,
    hf_y=noisy_sine,
    row=1,
    col=2,
)

# ------------- Power consumption data -------------
df_data_pc = df_data_pc.last("190D")
for i, c in enumerate(df_data_pc.columns):
    fig.add_trace(
        go.Scattergl(
            name=f"R-{i+1}", #line_width=1,
        ),
        hf_x=df_data_pc.index,
        hf_y=df_data_pc[c],
        row=2,
        col=1,
        # downsampler=LTTB(interleave_gaps=True),
    )

fig.update_layout(height=650)
fig.update_yaxes(title_text="Watt/hour", row=2, col=1)
fig.update_layout(
    title="<b>Plotly-Resampler demo</b>",
    title_x=0.5,
    legend_traceorder="normal",
    legend=dict(orientation="h", y=1.11, xanchor="left", x=0),
)

fig.show_dash(mode="external", debug=True, port=9029)

Dash app running on http://127.0.0.1:9029/


# Converting a `go.Figure`, with its traces, into a `FigureResampler`

This example first creates the `.gif` figure (with less data, otherwise the graph consruction time would be too long) and then uses the  `convert_existing_traces` argument of the FigureResampler constructor to convert this into a FigureResampler figure.

In [22]:
df_gusb = pd.read_parquet(f"data/df_gusb.parquet")
df_data_pc = pd.read_parquet(f"data/df_pc_test.parquet")

n = 110_000
x = np.arange(n)
noisy_sine = (3 + np.sin(x / 2_000) + np.random.randn(n) / 10) * x / 2_000

# construct a normal figure object instead of a figureResample object
fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{}, {}], [{"colspan": 2}, None]],
    subplot_titles=("GUSB swimming pool", "Generated sine", "Power consumption"),
    vertical_spacing=0.12,
)


# ------------ swimming pool data -----------
df_gusb_pool = df_gusb["zwembad"].last("4D").dropna()
fig.add_trace(
    go.Scattergl(
        x=df_gusb_pool.index,
        y=df_gusb_pool,  # .astype("uint16"),
        mode="markers",
        marker_size=5,
        name="occupancy",
        showlegend=True,
        hovertext="mean last hour: "
        + df_gusb_pool.rolling("1h").mean().astype(int).astype(str),
    ),
    # downsampler=EveryNthPoint(interleave_gaps=False),
    row=1,
    col=1,
)
fig.update_yaxes(title_text="Occupancy", row=1, col=1)


# ----------------- generated sine -----------
fig.add_trace(
    go.Scattergl(name="sin", line_color="#26b2e0", x=x, y=noisy_sine),
    row=1,
    col=2,
)

# ------------- Power consumption data -------------
df_data_pc = df_data_pc.last("190D")
for i, c in enumerate(df_data_pc.columns):
    fig.add_trace(
        go.Scattergl(
            name=f"room {i+1}",
            x=df_data_pc.index,
            y=df_data_pc[c],
        ),
        row=2,
        col=1,
    )

fig.update_layout(height=600)
fig.update_yaxes(title_text="Watt/hour", row=2, col=1)
fig.update_layout(
    title="<b>Plotly-Resampler demo - fig base</b>",
    title_x=0.5,
    legend_traceorder="normal",
    legend=dict(orientation="h", y=1.11, xanchor="left", x=0),
);

Note how the `data` property shape is the raw data size

In [23]:
fig.data[1]['x'].shape

(110000,)

In [24]:
# Convert the figure into a figurResampler figure by decorating it
fr_fig = FigureResampler(fig, default_n_shown_samples=500, convert_existing_traces=True)
fr_fig.show_dash(mode="external")

Dash app running on http://127.0.0.1:8050/


In [25]:
print('aggregated data    ', fr_fig.data[1]['x'].shape)
print('raw data (hf_data) ', fr_fig.hf_data[1]['x'].shape)

aggregated data     (500,)
raw data (hf_data)  (110000,)


In [26]:
# The data-shape of the original figure stil remains the same
fig.data[1]['x'].shape

(110000,)

**Note**:
* the data size of `fr_fig` is reduced to `default_n_shown_samples` (_500_) but the original `fig` data is still equal to `110_000`.
* the raw data can be accessed and altered using the `hf_data` of the FigureResampler object ⬇️


In [27]:
fr_fig.hf_data[1]['y'] = - noisy_sine ** 2

In [28]:
fr_fig.show_dash(mode='inline')

# Skin conductance example

This example is especially interesting as it uses a _background-color_ to indicate a signal quality.


In [29]:
df_gsr = pd.read_parquet('data/processed_gsr.parquet')

In [30]:
fig = FigureResampler(
    make_subplots(
        rows=2,
        cols=1,
        specs=[[{"secondary_y": True}], [{}]],
        shared_xaxes=True,
    ),
    default_n_shown_samples=1_000,
    verbose=False,
)
fig.update_layout(height=600, title='skin conductance example', title_x=0.5)


# -------------------------------- ROW 1 --------------------------------
# Add the skin conductance signals
for c in ["EDA", "EDA_lf_cleaned", "EDA_lf_cleaned_tonic"]:
    fig.add_trace(go.Scattergl(name=c), hf_x=df_gsr.index, hf_y=df_gsr[c], row=1, col=1)


# Show the Skin Conductance Response Peaks (SCR)
df_peaks = df_gsr[df_gsr["SCR_Peaks_neurokit_reduced_acc"] == 1]
fig.add_trace(
    trace=go.Scattergl(
        x=df_peaks.index,
        y=df_peaks["EDA_lf_cleaned"],
        visible="legendonly",
        mode="markers",
        marker_symbol="cross",
        marker_size=15,
        marker_color="red",
        name="SCR peaks",
    ),
    # Set limit_to_view to true so that the peaks dissapear when out-of view-range 
    # and thus not disturb the autoscale!!!
    limit_to_view=True,
)


# Display the Sking conductance Signal Quality As background 
df_grouped = groupby_consecutive(df_gsr["EDA_SQI"])
df_grouped["EDA_SQI"] = df_grouped["EDA_SQI"].map(bool)
df_grouped["good_sqi"] = df_grouped["EDA_SQI"].map(int)
df_grouped["bad_sqi"] = (~df_grouped["EDA_SQI"]).map(int)
for sqi_col, col_or in [
    ("good_sqi", "#2ca02c"),
    ("bad_sqi", "#d62728"),
]:
    fig.add_trace(
        go.Scattergl(
            x=df_grouped["start"],
            y=df_grouped[sqi_col],
            mode="lines",
            line_width=0,
            fill="tozeroy",
            fillcolor=col_or,
            opacity=0.1 if "good" in sqi_col else 0.2,
            line_shape="hv",
            name=sqi_col,
            showlegend=False,
        ),
        # The most important thing here is that interleave gaps is set to True
        # Additionally, the limit-to-view also ensures that the autoscale is not 
        # disturbed.
        downsampler=EveryNthPoint(interleave_gaps=False),
        limit_to_view=True,
        secondary_y=True,
    )


# -------------------------------- ROW 2 --------------------------------
# show the phasic EDA component
fig.add_trace(
    go.Scattergl(name="EDA_Phasic", visible="legendonly"),
    hf_x=df_gsr.index,
    hf_y=df_gsr["EDA_Phasic"],
    row=2,
    col=1,
)

fig.show_dash(mode="external", port=9022)

c:\ProgramData\Anaconda3\lib\site-packages\plotly_resampler\aggregation\aggregation_interface.py:97: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



NameError: name 'groupby_consecutive' is not defined

* The most important thing regarding this 

# Categorical series - box & histogram

This example highlights how `plotly-resampler` supports combining high-frequency trace-subplots 
with non-scatterlike traces such a a histogram & a boxplot.

In [ ]:
# Create a categorical series, with mostly a's, but a few sparse b's and c's
cats_list = np.array(list("aaaaaaaaaa" * 1000))
cats_list[np.random.choice(len(cats_list), 100, replace=False)] = "b"
cats_list[np.random.choice(len(cats_list), 50, replace=False)] = "c"
cat_series = pd.Series(cats_list, dtype="category",)

_nb_samples = 30_000
x = np.arange(_nb_samples).astype(np.uint32)
y = np.sin(x / 300).astype(np.float32) + np.random.randn(_nb_samples) / 5
float_series = pd.Series(index=x, data=y)

In [ ]:
base_fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{}, {}], [{"colspan": 2}, None]],
)
fig = FigureResampler(base_fig, default_n_shown_samples=1000, verbose=False)

fig.add_trace(
    go.Scattergl(name="cat_series"),
    hf_x=cat_series.index,
    hf_y=cat_series,
    row=1,
    col=1,
    hf_hovertext="text",
)

fig.add_trace(go.Box(x=float_series.values, name="float_series"), row=1, col=2)
fig.add_trace(
    go.Box(x=float_series.values ** 2, name="float_series**2"), row=1, col=2
)

# add a not hf-trace
fig.add_trace(
    go.Histogram(
        x=float_series,
        name="float_series",
    ),
    row=2,
    col=1,
)
fig.show_dash(mode="external", port=9023)